In [4]:
import networkx as nx
import itertools
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import heapq


In [18]:
def generate_binary_labels(c: tuple, n: int):
    return [format(i, '0' + str(n) + 'b') for i in c]

def hamming_distance(s1, s2):
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))

def create_graph(labels):
    G = nx.Graph()
    for label in labels:
        G.add_node(label)
    for u, v in itertools.combinations(labels, 2):
        distance = hamming_distance(u, v)
        G.add_edge(u, v, weight=distance)
    return G

def total_weight(G):
    return sum(weight for u, v, weight in G.edges.data('weight'))

def visualize_graph_and_mst(w, idx, grid_size, fig):
    pos = nx.planar_layout(w.mst)  
    ax = fig.add_subplot(grid_size, grid_size, idx)
    
    plt.axis('off') 
    
    nx.draw(w.mst, pos, with_labels=True, node_size=100, node_color='lightblue', edge_color='gray', alpha=0.6, ax=ax)
    # nx.draw_networkx_edges(w.graph, pos, edgelist=w.mst.edges(), edge_color='red', width=2, ax=ax)
    nx.draw_networkx_edge_labels(w.mst, pos, edge_labels=nx.get_edge_attributes(w.mst, 'weight'))
    
    ax.set_title(f"Weight: {w.weight}", fontsize=10)  # Use the weight from the graph attribute


class wrapper:
    def __init__(self, w, m, g):
        self.weight = w
        self.mst = m
        self.graph = G
    
    def __lt__(self, other):
        return self.weight < other.weight

    def __eq__(self, other):
        return self.weight == other.weight
    
    def __str__(self):
        return f"{self.weight}"

In [20]:
n = 3
t = 12
pdf = PdfPages(f'graph_visualizations_{n}.pdf')
for k in range(4, 6):
    heap = []
    for combo in itertools.combinations([_ for _ in range(1, 2 ** n)], k - 1):
        combo = 0, *combo
        labels = generate_binary_labels(combo, n)
        G = create_graph(labels)
        mst = nx.minimum_spanning_tree(G)
        mst_weight = total_weight(mst)
        heapq.heappush(heap, wrapper(mst_weight, mst, G))
        if len(heap) > t:
            heapq.heappop(heap)

    top = sorted(heap, key=lambda x: x.weight, reverse=True)
    grid_size = 4

    fig = plt.figure(figsize=(grid_size * 5, grid_size * 4)) 
    plt.clf()
    for i, w in enumerate(top, start=1):
        visualize_graph_and_mst(w, i, grid_size, fig)

    plt.tight_layout()
    plt.suptitle(f'Graphs {k} vertices {n} qubits', fontsize=16)
    pdf.savefig(fig)
    plt.close()

pdf.close()